<a href="https://colab.research.google.com/github/ozturkergin/ozturkergin/blob/main/TEFAS_PowerBI_Import_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
pip install bs4


In [8]:
pip install urllib3

In [9]:
pip install marshmallow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 748.8 kB/s eta 0:00:00


In [10]:
import requests
import pandas as pd
import math
import ssl
import concurrent.futures
import time

from datetime import datetime, timedelta, date
from typing import Dict, List, Optional, Union
from marshmallow import Schema, fields, EXCLUDE, pre_load, post_load
from requests.adapters import HTTPAdapter
from bs4 import BeautifulSoup
from urllib3.poolmanager import PoolManager
from urllib3.util.retry import Retry

# Special thanks to https://github.com/burakyilmaz321

class InfoSchema(Schema):
    code = fields.String(data_key="FONKODU", allow_none=True)
    fonunvantip = fields.String(data_key="FONUNVANTIP", allow_none=True)
    date = fields.Date(data_key="TARIH", allow_none=True)
    price = fields.Float(data_key="FIYAT", allow_none=True)
    title = fields.String(data_key="FONUNVAN", allow_none=True)
    market_cap = fields.Float(data_key="PORTFOYBUYUKLUK", allow_none=True)
    number_of_shares = fields.Float(data_key="TEDPAYSAYISI", allow_none=True)
    number_of_investors = fields.Float(data_key="KISISAYISI", allow_none=True)

    @pre_load
    def pre_load_hook(self, input_data, **kwargs):
        # Convert milliseconds Unix timestamp to date
        seconds_timestamp = int(input_data["TARIH"]) / 1000
        input_data["TARIH"] = date.fromtimestamp(seconds_timestamp).isoformat()
        return input_data

    @post_load
    def post_load_hool(self, output_data, **kwargs):
        # Fill missing fields with default None
        output_data = {f: output_data.setdefault(f) for f in self.fields}
        return output_data

    class Meta:
        unknown = EXCLUDE

class tefas_get:

    root_url = "https://www.tefas.gov.tr"
    #detail_endpoint = "/api/DB/BindHistoryAllocation"
    info_endpoint = "/api/DB/BindHistoryInfo"
    headers = {
        "Connection": "keep-alive",
        "X-Requested-With": "XMLHttpRequest",
        "User-Agent": (
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 "
            "(KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36"
        ),
        "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
        "Accept": "application/json, text/javascript, */*; q=0.01",
        "Origin": "https://www.tefas.gov.tr",
        "Referer": "https://www.tefas.gov.tr/TarihselVeriler.aspx",
    }

    @staticmethod
    def get_combobox_items(url, select_id):
        # Send a GET request to the URL
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code != 200:
            raise Exception(f"Failed to fetch the URL: {response.status_code}")

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the select element by its ID
        select_element = soup.find('select', id=select_id)

        # Check if the select element exists
        if not select_element:
            raise Exception(f"Select element with id '{select_id}' not found")

        # Get all option elements within the select element
        options = select_element.find_all('option')
        options = list(filter(None, options))

        # Extract the text and value for each option
        items = []
        for option in options:
            value = option.get('value')
            items.append(value)

        items.remove('')
        items.remove('Serbest')

        return items

    def fetch_info(self, fonunvantip, start_date_initial, end_date_initial ):

        counter = 1
        start_date = start_date_initial
        end_date = end_date_initial
        range_date = end_date_initial - start_date_initial
        range_interval = 90
        info_schema = InfoSchema(many=True)
        info_result = pd.DataFrame()

        if range_date.days > range_interval :
           counter = range_date.days / range_interval
           counter = math.ceil(counter)
           end_date = start_date + timedelta(days=range_interval)

        while counter > 0:
           counter -= 1

           data = {
                    "fontip": "YAT",
                    "bastarih": self._parse_date(start_date),
                    "bittarih": self._parse_date(end_date),
                    "fonunvantip": fonunvantip,
                    "fonkod": "",
                  }

           info = self._do_post(self.info_endpoint, data)
           info = info_schema.load(info)
           info = pd.DataFrame(info, columns=info_schema.fields.keys())
           info['fonunvantip'] = fonunvantip
           info = info[info['price'] != 0]
           info_result = pd.concat([info_result, info])
           info_result = info_result.reset_index(drop=True)
           info = info.reset_index(drop=True)

           if counter > 0 :
              start_date = end_date + timedelta(days=1)
              end_date = end_date + timedelta(days=range_interval)
              if end_date > end_date_initial :
                 end_date = end_date_initial

        #print(f"Data for {fonunvantip}: {len(info_result)} rows  {start_date_initial} {end_date_initial} counter {counter}" )
        return info_result

    def fetch_info_concurrently(self, fonunvantips, start_date_initial, end_date_initial):
        with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
            merged = pd.DataFrame()
            # Submit all tasks to the executor
            futures = {executor.submit(self.fetch_info, fonunvantip, start_date_initial, end_date_initial): fonunvantip for fonunvantip in fonunvantips}

            # Retrieve results as they complete
            for future in concurrent.futures.as_completed(futures):
                info = future.result()
                merged = pd.concat([merged, info])
                #print(f"Data for merged: {fonunvantip} {len(merged)} info {len(info)}" )

            return merged

    def fetch(
        self,
        start: Union[str, datetime],
        end: Optional[Union[str, datetime]] = None,
        columns: Optional[List[str]] = None,
		concurrent = False
    ) -> pd.DataFrame:

        start_date_initial = datetime.strptime(start, "%Y-%m-%d")
        end_date_initial = datetime.strptime(end or start, "%Y-%m-%d")

        merged = pd.DataFrame()
        if concurrent != False :
            fonunvantips = self.get_combobox_items(url="https://www.tefas.gov.tr/TarihselVeriler.aspx", select_id="DropDownListFundTypeExplanationYAT")
            merged = self.fetch_info_concurrently(fonunvantips, start_date_initial, end_date_initial)
        else :
            merged = self.fetch_info("", start_date_initial, end_date_initial)

        merged = merged[columns] if columns and not merged.empty else merged

        return merged

    def _do_post(self, endpoint: str, data: Dict[str, str]) -> Dict[str, str]:
        time.sleep(1)
        session = requests.Session()
        retry_strategy = Retry(
            total=3,  # Total number of retries
            backoff_factor=1,  # Backoff factor for retries
            status_forcelist=[429, 500, 502, 503, 504],  # Retry on these status codes
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        session.mount("https://", adapter)

        self.cookies = session.cookies.get_dict()

        response = session.post(
            url=f"{self.root_url}/{endpoint}",
            data=data,
            cookies=self.cookies,
            headers=self.headers,
        )
        # Check the response status code and content
        if response.status_code != 200:
            print(f"Request failed with status code: {response.status_code}")
            print(f"Response content: {response.text}")
            return {}  # Return an empty dictionary if the request failed
        try:
            return response.json().get("data", {})
        except ValueError as e:
            print(f"Error decoding JSON response: {e}")
            print(f"Response content: {response.text}")
            return {}

    def _parse_date(self, date: Union[str, datetime]) -> str:
        if isinstance(date, datetime):
            formatted = datetime.strftime(date, "%d.%m.%Y")
        elif isinstance(date, str):
            try:
                parsed = datetime.strptime(date, "%Y-%m-%d")
            except ValueError as exc:
                raise ValueError(
                    "Date string format is incorrect. " "It should be `YYYY-MM-DD`"
                ) from exc
            else:
                formatted = datetime.strftime(parsed, "%d.%m.%Y")
        else:
            raise ValueError(
                "`date` should be a string like 'YYYY-MM-DD' "
                "or a `datetime.datetime` object."
            )
        return formatted

def calculate_rsi(group, window_length):
    # Calculate the daily price changes
    group = group.sort_values('date', ascending=True)
    group['price_change'] = group['price'].diff()

    # Separate the gains and losses
    group['gain'] = group['price_change'].apply(lambda x: x if x > 0 else 0)
    group['loss'] = group['price_change'].apply(lambda x: abs(x) if x < 0 else 0)

    # Calculate the average gain and loss using the exponential moving average
    group['avg_gain'] = group['gain'].rolling(window=window_length, min_periods=1).mean()
    group['avg_loss'] = group['loss'].rolling(window=window_length, min_periods=1).mean()

    group = group.reset_index(drop=True)  # This line is added

    group.loc[group['avg_loss'] == 0, 'rs'] = 0
    group.loc[group['avg_loss'] == 0, 'rsi'] = 100

    # Calculate the Relative Strength (RS)
    group.loc[group['avg_loss'] != 0, 'rs'] = group['avg_gain'] / group['avg_loss']

    # Calculate the RSI
    group.loc[group['avg_loss'] != 0, 'rsi'] = 100 - (100 / (1 + group['rs']))

    # Forward fill RSI values to ensure we have RSI for each day
    group['rsi'] = group['rsi'].fillna(method='ffill')

    group.iloc[:window_length, group.columns.get_loc('rs')] = 0
    group.iloc[:window_length, group.columns.get_loc('rsi')] = 0
    group = group.sort_values('date', ascending=False)

    return group

tefas = tefas_get()

time_delta = 366
start_date_calc = date.today() - timedelta(days=time_delta)
date_start = start_date_calc.strftime("%Y-%m-%d")
date_end = date.today().strftime("%Y-%m-%d")

fetched_data = pd.DataFrame()
fetched_data = tefas.fetch(start=date_start, end=date_end, columns=["code", "date", "price", "fonunvantip", "title", "market_cap", "number_of_shares", "number_of_investors" ], concurrent=True)
fetched_data['date'] = pd.to_datetime(fetched_data['date'], errors='coerce')
fetched_data['date'].dt.strftime('%Y-%m-%d')
fetched_data['date'] = fetched_data['date'].dt.date
fetched_data['price'].astype(float,False)
fetched_data[(fetched_data!=0)&(pd.isnull(fetched_data))]

# Group by stock and apply the RSI calculation
window_length = 14
fetched_data = fetched_data.groupby(['code','fonunvantip']).apply(calculate_rsi, window_length=window_length).reset_index(drop=True)

fetched_data = fetched_data.fillna(0)
fetched_data['rsi'].astype(float,False)
fetched_data['avg_loss'].astype(float,False)
fetched_data['avg_gain'].astype(float,False)
fetched_data['gain'].astype(float,False)
fetched_data['loss'].astype(float,False)
fetched_data['price_change'].astype(float,False)
fetched_data['market_cap'].astype(float,False)
fetched_data['number_of_shares'].astype(float,False)
fetched_data['number_of_investors'].astype(float,False)


Error decoding JSON response: Expecting value: line 1 column 1 (char 0)
Response content: <html><head><title>Request Rejected</title></head><body>The requested URL was rejected. Please consult with your administrator.<br><br>Your support ID is: <  891378466735339548><br><br><a href='javascript:history.back();'>[Go Back]</body></html>
Error decoding JSON response: Expecting value: line 1 column 1 (char 0)
Response content: <html><head><title>Request Rejected</title></head><body>The requested URL was rejected. Please consult with your administrator.<br><br>Your support ID is: <  891378466737411019><br><br><a href='javascript:history.back();'>[Go Back]</body></html>
Error decoding JSON response: Expecting value: line 1 column 1 (char 0)
Response content: <html><head><title>Request Rejected</title></head><body>The requested URL was rejected. Please consult with your administrator.<br><br>Your support ID is: <  891378466734697197><br><br><a href='javascript:history.back();'>[Go Back]</body>

0         903.0
1         903.0
2         900.0
3         901.0
4         905.0
          ...  
100351      1.0
100352      1.0
100353      1.0
100354      1.0
100355      1.0
Name: number_of_investors, Length: 100356, dtype: float64

In [11]:
fetched_data

,code,date,price,fonunvantip,title,market_cap,number_of_shares,number_of_investors,price_change,gain,loss,avg_gain,avg_loss,rs,rsi
0,AAK,2024-04-09,19.975836,Değişken,ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON,1.081857e+08,5415830.0,903.0,0.134933,0.134933,0.000000,0.116439,0.023350,4.986592,83.296007
1,AAK,2024-04-08,19.840903,Değişken,ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON,1.073510e+08,5410589.0,903.0,0.487520,0.487520,0.000000,0.120318,0.023350,5.152726,83.747042
2,AAK,2024-04-05,19.353383,Değişken,ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON,1.046162e+08,5405579.0,900.0,0.195257,0.195257,0.000000,0.085495,0.029946,2.854997,74.059646
3,AAK,2024-04-04,19.158126,Değişken,ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON,1.045092e+08,5455084.0,901.0,-0.016536,0.000000,0.016536,0.071548,0.033494,2.136164,68.113911
4,AAK,2024-04-03,19.174662,Değişken,ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON,1.050448e+08,5478315.0,905.0,-0.066382,0.000000,0.066382,0.071548,0.040082,1.785046,64.093950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100351,ZSN,2023-07-25,26.497558,Döviz Cinsinden İhraç (Dolar),ZİRAAT PORTFÖY ÖPY BEŞİNCİ SERBEST (DÖVİZ-ABD ...,3.433818e+05,12959.0,1.0,0.025998,0.025998,0.000000,0.020648,0.016886,0.000000,0.000000
100352,ZSN,2023-07-24,26.471560,Döviz Cinsinden İhraç (Dolar),ZİRAAT PORTFÖY ÖPY BEŞİNCİ SERBEST (DÖVİZ-ABD ...,3.430449e+05,12959.0,1.0,0.006253,0.006253,0.000000,0.019310,0.021107,0.000000,0.000000
100353,ZSN,2023-07-21,26.465307,Döviz Cinsinden İhraç (Dolar),ZİRAAT PORTFÖY ÖPY BEŞİNCİ SERBEST (DÖVİZ-ABD ...,3.429639e+05,12959.0,1.0,-0.084429,0.000000,0.084429,0.023662,0.028143,0.000000,0.000000
100354,ZSN,2023-07-20,26.549736,Döviz Cinsinden İhraç (Dolar),ZİRAAT PORTFÖY ÖPY BEŞİNCİ SERBEST (DÖVİZ-ABD ...,3.440580e+05,12959.0,1.0,0.070987,0.070987,0.000000,0.035493,0.000000,0.000000,0.000000


In [ ]:
fetched_data.to_csv('tefas.csv', index=False)

In [12]:
# prompt: fetched_data dataframe google sheets olarak kaydet

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# Create a new spreadsheet
sh = gc.create('Tefas Data')

# Open the first worksheet
worksheet = gc.open('Tefas Data').sheet1

# Clear any existing data in the worksheet
worksheet.clear()

# Convert the DataFrame to a list of lists
data = fetched_data.values.tolist()

# Insert the data into the worksheet
for row in data:
    for i in range(len(row)):
        if isinstance(row[i], date):
            row[i] = row[i].strftime('%Y-%m-%d')  # Format date objects as strings

# Insert the data into the worksheet
worksheet.insert_rows(data)


{'spreadsheetId': '1mlKRGUKXimJrjHmzVnzRIqkCLr4YmLcr4NSzhgoArEw',
 'updates': {'spreadsheetId': '1mlKRGUKXimJrjHmzVnzRIqkCLr4YmLcr4NSzhgoArEw',
  'updatedRange': 'Sayfa1!A1:O100356',
  'updatedRows': 100356,
  'updatedColumns': 15,
  'updatedCells': 1505340}}